# Использование хаба промптов для суммаризации книг
В качестве примера суммаризируем первые две главы книги Булгакова "Мастер и Маргарита".
## Инициализация GigaChat
> Обратите внимание, что нужно использовать большую модель гигачат, поддерживающую 8 тысяч токенов и более!
> Обратите внимание, что если в книге могут встречаться слова и темы, которые ГигаЧат может посчтитать неоднозначными,
> следует использова флаг отключения фильтрации - profanity=False. По возможности использования этого флага следует обратиться 
> в поддержку GigaChat. На данный момент эта опция доступна только юридическим лицам.

In [31]:
from langchain.prompts import load_prompt
from langchain.chains.summarize import load_summarize_chain
from langchain_community.chat_models import GigaChat

giga = GigaChat(credentials=..., model=...big_model..., verify_ssl_certs=False, profanity=False)

## Суммаризация книг, больших текстов
### Загрузка и подготовка данных

In [27]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = TextLoader("../../../../docs/docs/use_cases/мастер_и_маргарита.txt")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 7000,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = False,
)
documents = text_splitter.split_documents(documents)
print(f"Количество частей книги: {len(documents)}")

Количество частей книги: 11


### Создание цепочки суммаризации

In [32]:
book_map_prompt = load_prompt("lc://prompts/summarize/map_reduce/summarize_book_map.yaml")
book_combine_prompt = load_prompt("lc://prompts/summarize/map_reduce/summarize_book_combine.yaml")

chain = load_summarize_chain(giga, chain_type="map_reduce", 
                             map_prompt=book_map_prompt,
                             combine_prompt=book_combine_prompt,
                             verbose=False)

In [33]:
giga.verbose = True
res = chain.invoke({"input_documents": documents})

Giga request: {"messages": [{"role": "user", "content": "Напиши краткое содержимое части книги объемом в 2-3 предложения.\n\n\"Михаил Булгаков\n\nМастер и Маргарита\n\nМосква 1984\n\n\nТекст печатается в последней прижизненной редакции (рукописи хранятся в рукописном отделе Государственной библиотеки СССР имени В. И. Ленина), а также с исправлениями и дополнениями, сделанными под диктовку писателя его женой, Е. С. Булгаковой.\n\n\n\nЧАСТЬ ПЕРВАЯ\n\n\t\t...Так кто ж ты, наконец?\n\t\t– Я – часть той силы,\n\t\tчто вечно хочет\n\t\tзла и вечно совершает благо.\n\n    Гете. «Фауст»\n\n\n\nГлава 1\n\nНикогда не разговаривайте с неизвестными\n\nОднажды весною, в час небывало жаркого заката, в Москве, на Патриарших прудах, появились два гражданина. Первый из них, одетый в летнюю серенькую пару, был маленького роста, упитан, лыс, свою приличную шляпу пирожком нес в руке, а на хорошо выбритом лице его помещались сверхъестественных размеров очки в черной роговой оправе. Второй – плечистый, рыже

In [22]:
print(res["output_text"].replace(". ", ".\n"))

Роман "Мастер и Маргарита" Михаила Булгакова начинается с описания встречи двух писателей, Михаила Берлиоза и Ивана Бездомного, на Патриарших прудах в Москве.
Они обсуждают религиозные темы, включая существование Иисуса Христа, и Берлиоз утверждает, что Иисус является мифом, не имеющим исторической основы.
В этот момент появляется загадочный гражданин в клетчатом пиджаке, который вызывает у Берлиоза чувство страха и тревоги.
В ходе беседы с писателями, иностранец обсуждает вопросы веры, судьбы и смерти, утверждая, что без веры в высшую силу жизнь человека становится бессмысленной и непредсказуемой.

Роман также описывает встречу прокуратора Иудеи Понтия Пилата с арестованным Иешуа Га-Ноцри, обвиняемым в подстрекательстве к разрушению храма.
Иешуа отрицает обвинения, утверждая, что его слова были неправильно истолкованы.
Пилат, страдающий от головной боли, пытается разобраться в ситуации, задавая вопросы и пытаясь понять мотивы Иешуа.

Прокуратор Иудеи Пилат разбирает дело бродячего фил

Если нужно изменить количество предложений в суммаризации, то с помощью дополнительных параметров можно задать нужные объемы текста на этапах map и reduce.

In [23]:
res = chain.invoke({"input_documents": documents, "map_size": "одно предложение", "combine_size": "три предложения"})

In [24]:
print(res["output_text"].replace(". ", ".\n"))

В первой части романа "Мастер и Маргарита" Михаил Булгаков описывает встречу двух писателей на Патриарших прудах, где они обсуждают существование Иисуса Христа.
Неизвестный иностранец предсказывает Берлиозу и Бездомному, что заседание МАССОЛИТа не состоится из-за того, что Аннушка уже купила подсолнечное масло и разлила его.
Прокуратор Пилат допрашивает арестованного Иешуа Га-Ноцри, обсуждая его высказывания о храме и его связь с сборщиком податей Левием Матвеем.
